In [1]:
# Importing Libraries 
import pandas as pd
import numpy as np

In [2]:
# Read data
order_report = pd.read_excel('Company X - Order Report.xlsx')
sku_master = pd.read_excel('Company X - SKU Master.xlsx')
pincode_mapping = pd.read_excel('Company X - Pincode Zones.xlsx')
courier_invoice = pd.read_excel('Courier Company - Invoice.xlsx')
courier_company_rates = pd.read_excel('Courier Company - Rates.xlsx')

In [3]:
# Preview of dataset
print("Order Report:")
print(order_report.head())
print("\nSKU Master:")
print(sku_master.head())
print("\nPincode Mapping:")
print(pincode_mapping.head())
print("\nCourier Invoice:")
print(courier_invoice.head())
print("\nCourier Company rates:")
print(courier_company_rates.head())

Order Report:
   ExternOrderNo            SKU  Order Qty
0     2001827036  8904223818706        1.0
1     2001827036  8904223819093        1.0
2     2001827036  8904223819109        1.0
3     2001827036  8904223818430        1.0
4     2001827036  8904223819277        1.0

SKU Master:
             SKU  Weight (g)
0  8904223815682         210
1  8904223815859         165
2  8904223815866         113
3  8904223815873          65
4  8904223816214         120

Pincode Mapping:
   Warehouse Pincode  Customer Pincode Zone
0             121003            507101    d
1             121003            486886    d
2             121003            532484    d
3             121003            143001    b
4             121003            515591    d

Courier Invoice:
        AWB Code    Order ID  Charged Weight  Warehouse Pincode  \
0  1091117222124  2001806232            1.30             121003   
1  1091117222194  2001806273            1.00             121003   
2  1091117222931  2001806408            

In [4]:
# Check for missing values
print("\nMissing values in Website Order Report:")
print(order_report.isnull().sum())
print("\nMissing values in SKU Master:")
print(sku_master.isnull().sum())
print("\nMissing values in Pincode Mapping:")
print(pincode_mapping.isnull().sum())
print("\nMissing values in Courier Invoice:")
print(courier_invoice.isnull().sum())
print("\nMissing values in courier company rates:")
print(courier_company_rates.isnull().sum())


Missing values in Website Order Report:
ExternOrderNo    0
SKU              0
Order Qty        0
dtype: int64

Missing values in SKU Master:
SKU           0
Weight (g)    0
dtype: int64

Missing values in Pincode Mapping:
Warehouse Pincode    0
Customer Pincode     0
Zone                 0
dtype: int64

Missing values in Courier Invoice:
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

Missing values in courier company rates:
Zone                                     0
Weight Slabs                             0
Forward Fixed Charge                     0
Forward Additional Weight Slab Charge    0
RTO Fixed Charge                         0
RTO Additional Weight Slab Charge        0
dtype: int64


In [5]:
# Merge the Order Report and SKU Master based on SKU
merged_data = pd.merge(order_report, sku_master, on='SKU')
print(merged_data.head())

   ExternOrderNo            SKU  Order Qty  Weight (g)
0     2001827036  8904223818706        1.0         127
1     2001821995  8904223818706        1.0         127
2     2001819252  8904223818706        1.0         127
3     2001816996  8904223818706        1.0         127
4     2001814580  8904223818706        1.0         127


In [6]:
# Rename the "ExternOrderNo" column to "Order ID" in the merged_data DataFrame
merged_data = merged_data.rename(columns={'ExternOrderNo': 'Order ID'})

In [7]:
# create a new DataFrame called “one_courier” to store informatio Extract the unique customer pin codes from the pincode mapping dataset
one_courier = pincode_mapping.drop_duplicates(subset=['Customer Pincode'])

In [8]:
# merge the courier invoice and pincode mapping dataset:
courier_abc= courier_invoice[['Order ID', 'Customer Pincode','Type of Shipment']]
pincodes= courier_abc.merge(one_courier,on='Customer Pincode')
print(pincodes.head())

     Order ID  Customer Pincode Type of Shipment  Warehouse Pincode Zone
0  2001806232            507101  Forward charges             121003    d
1  2001806273            486886  Forward charges             121003    d
2  2001806408            532484  Forward charges             121003    d
3  2001806458            143001  Forward charges             121003    b
4  2001807012            515591  Forward charges             121003    d


In [9]:
# Now let’s merge the pin codes with the main dataframe:
merged2 = merged_data.merge(pincodes, on='Order ID')

In [10]:
merged2.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e


In [11]:
# calculate the weight in kilograms by dividing the ‘Weight (g)’ column in the ‘merged2’ DataFrame by 1000:
merged2['Weights (Kgs)'] = merged2['Weight (g)'] / 1000

In [12]:
# calculate the weight slabs:
def weight_slab(weight):
    i = round(weight % 1, 1)
    if i == 0.0:
        return weight
    elif i > 0.5:
        return int(weight) + 1.0
    else:
        return int(weight) + 0.5

merged2['Weight Slab (KG)'] = merged2['Weights (Kgs)'].apply(weight_slab)
courier_invoice['Weight Slab Charged by Courier Company']=(courier_invoice['Charged Weight']).apply(weight_slab)

In [13]:
courier_invoice.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Weight Slab Charged by Courier Company
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.5
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,1.0
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.5
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,1.0
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.5


In [14]:
merged2.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Weights (Kgs),Weight Slab (KG)
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5


In [15]:
# Now let’s rename the columns to prepare the desired dataframe:
courier_invoice = courier_invoice.rename(columns={'Zone': 'Delivery Zone Charged by Courier Company'})
merged2 = merged2.rename(columns={'Zone': 'Delivery Zone As Per X'})
merged2 = merged2.rename(columns={'Weight Slab (KG)': 'Weight Slab As Per X'})

In [16]:
merged2.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per X,Weights (Kgs),Weight Slab As Per X
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5


In [17]:
courier_company_rates.head()

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0.50,29.5,23.6,13.6,23.6
1,B,1.00,33.0,28.3,20.5,28.3
2,C,1.25,40.1,38.9,31.9,38.9
3,D,1.50,45.4,44.8,41.3,44.8
4,E,2.00,56.6,55.5,50.7,55.5


In [18]:
# Now let’s calculate the expected charges:
total_expected_charge = []

for _, row in merged2.iterrows():
    fwd_fixed = courier_company_rates.at[0, 'Forward Fixed Charge']
    fwd_additional = courier_company_rates.at[0, 'Forward Additional Weight Slab Charge']
    rto_fixed = courier_company_rates.at[0, 'RTO Fixed Charge']
    rto_additional = courier_company_rates.at[0, 'RTO Additional Weight Slab Charge']

    weight_slab = row['Weight Slab As Per X']

    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
    else:
        total_expected_charge.append(0)

merged2['Expected Charge as per X'] = total_expected_charge
print(merged2.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8904223818706        1.0         127            173213   
1  2001827036  8904223819093        1.0         150            173213   
2  2001827036  8904223819109        1.0         100            173213   
3  2001827036  8904223818430        1.0         165            173213   
4  2001827036  8904223819277        1.0         350            173213   

  Type of Shipment  Warehouse Pincode Delivery Zone As Per X  Weights (Kgs)  \
0  Forward charges             121003                      e          0.127   
1  Forward charges             121003                      e          0.150   
2  Forward charges             121003                      e          0.100   
3  Forward charges             121003                      e          0.165   
4  Forward charges             121003                      e          0.350   

   Weight Slab As Per X  Expected Charge as per X  
0                   0.5           

In [19]:
merged2.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per X,Weights (Kgs),Weight Slab As Per X,Expected Charge as per X
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5,29.5
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5,29.5
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5,29.5
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5,29.5
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5,29.5


In [20]:
merged2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401 entries, 0 to 400
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Order ID                  401 non-null    int64  
 1   SKU                       401 non-null    object 
 2   Order Qty                 401 non-null    float64
 3   Weight (g)                401 non-null    int64  
 4   Customer Pincode          401 non-null    int64  
 5   Type of Shipment          401 non-null    object 
 6   Warehouse Pincode         401 non-null    int64  
 7   Delivery Zone As Per X    401 non-null    object 
 8   Weights (Kgs)             401 non-null    float64
 9   Weight Slab As Per X      401 non-null    float64
 10  Expected Charge as per X  401 non-null    float64
dtypes: float64(4), int64(4), object(3)
memory usage: 37.6+ KB


In [21]:
# calculate the Total Weight as per courier company (Kg):
total_weight_courier = []

for _, row in merged2.iterrows():
    weight_slab = row['Weight Slab As Per X']
    weight_of_shipment = row['Weights (Kgs)']
    
    # Calculate the total weight as per the courier company
    total_weight_courier.append(weight_of_shipment * weight_slab)

# Add the calculated total weight as per the courier company to the DataFrame
merged2['Total Weight as per Courier Company (KG)'] = total_weight_courier

In [22]:
merged2.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per X,Weights (Kgs),Weight Slab As Per X,Expected Charge as per X,Total Weight as per Courier Company (KG)
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5,29.5,0.0635
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5,29.5,0.0750
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5,29.5,0.0500
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5,29.5,0.0825
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5,29.5,0.1750


In [23]:
def get_weight_slab_charge(delivery_zone, weight_slab, courier_company_rates):
    # Assuming courier_company_rates is a DataFrame containing charge rates
    # This function retrieves the charge rate based on the delivery zone and weight slab
    
    # Retrieve charge rates from the courier company rates DataFrame
    fwd_fixed = courier_company_rates.at[0, 'Forward Fixed Charge']
    fwd_additional = courier_company_rates.at[0, 'Forward Additional Weight Slab Charge']
    rto_fixed = courier_company_rates.at[0, 'RTO Fixed Charge']
    rto_additional = courier_company_rates.at[0, 'RTO Additional Weight Slab Charge']
    
    # Calculate the weight slab charge based on the delivery type (Forward or Forward and RTO)
    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        return fwd_fixed + additional_weight * fwd_additional
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        return fwd_fixed + additional_weight * (fwd_additional + rto_additional)
    else:
        return 0  # Return 0 if shipment type is not specified or invalid

# Create an empty list to store weight slab charges
weight_slab_charge = []

# Iterate over each row in the DataFrame merged2
for _, row in merged2.iterrows():
    # Retrieve delivery zone and weight slab from the current row
    delivery_zone = row['Delivery Zone As Per X']
    weight_slab = row['Weight Slab As Per X']
    
    # Calculate the weight slab charge using the get_weight_slab_charge function
    charge = get_weight_slab_charge(delivery_zone, weight_slab, courier_company_rates)
    
    # Append the calculated charge to the weight_slab_charge list
    weight_slab_charge.append(charge)

# Add the weight slab charge as a new column to the DataFrame merged2
merged2['Weight Slab Charge by Courier Company'] = weight_slab_charge

# Print the DataFrame to see the results
print(merged2.head())


     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8904223818706        1.0         127            173213   
1  2001827036  8904223819093        1.0         150            173213   
2  2001827036  8904223819109        1.0         100            173213   
3  2001827036  8904223818430        1.0         165            173213   
4  2001827036  8904223819277        1.0         350            173213   

  Type of Shipment  Warehouse Pincode Delivery Zone As Per X  Weights (Kgs)  \
0  Forward charges             121003                      e          0.127   
1  Forward charges             121003                      e          0.150   
2  Forward charges             121003                      e          0.100   
3  Forward charges             121003                      e          0.165   
4  Forward charges             121003                      e          0.350   

   Weight Slab As Per X  Expected Charge as per X  \
0                   0.5          

In [35]:
charges_billed = []

for _, row in merged2.iterrows():
    weight_slab_charge = row['Weight Slab Charge by Courier Company']
    delivery_zone = row['Delivery Zone As Per X']
    weight_slab = row['Weight Slab As Per X']
    
    # Determine the charges based on the type of shipment (Forward or Forward and RTO)
    if row['Type of Shipment'] == 'Forward charges':
        # Calculate charges based on weight slab charge
        charges_billed.append(weight_slab_charge)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        # Calculate charges based on weight slab charge for Forward and RTO charges
        # You might need to adjust this calculation based on the courier company's invoice structure
        charges_billed.append(weight_slab_charge * 2)  # Assuming same charge for Forward and RTO
    else:
        charges_billed.append(0)  # Set charges to 0 for other types of shipments

# Add the calculated charges billed by the courier company to the DataFrame
merged2['Charges Billed by Courier Company'] = charges_billed

# Print the DataFrame to see the results
print(merged2.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8904223818706        1.0         127            173213   
1  2001827036  8904223819093        1.0         150            173213   
2  2001827036  8904223819109        1.0         100            173213   
3  2001827036  8904223818430        1.0         165            173213   
4  2001827036  8904223819277        1.0         350            173213   

  Type of Shipment  Warehouse Pincode Delivery Zone As Per X  Weights (Kgs)  \
0  Forward charges             121003                      e          0.127   
1  Forward charges             121003                      e          0.150   
2  Forward charges             121003                      e          0.100   
3  Forward charges             121003                      e          0.165   
4  Forward charges             121003                      e          0.350   

   Weight Slab As Per X  Expected Charge as per X  \
0                   0.5          

In [36]:
merged2.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per X,Weights (Kgs),Weight Slab As Per X,Expected Charge as per X,Total Weight as per Courier Company (KG),Weight Slab Charge by Courier Company,Total Weight as per X (Kg),Charges Billed by Courier Company
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5,29.5,0.0635,29.5,2.039,29.5
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5,29.5,0.0750,29.5,2.039,29.5
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5,29.5,0.0500,29.5,2.039,29.5
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5,29.5,0.0825,29.5,2.039,29.5
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5,29.5,0.1750,29.5,2.039,29.5


In [37]:
merged2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401 entries, 0 to 400
Data columns (total 15 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Order ID                                  401 non-null    int64  
 1   SKU                                       401 non-null    object 
 2   Order Qty                                 401 non-null    float64
 3   Weight (g)                                401 non-null    int64  
 4   Customer Pincode                          401 non-null    int64  
 5   Type of Shipment                          401 non-null    object 
 6   Warehouse Pincode                         401 non-null    int64  
 7   Delivery Zone As Per X                    401 non-null    object 
 8   Weights (Kgs)                             401 non-null    float64
 9   Weight Slab As Per X                      401 non-null    float64
 10  Expected Charge as per X              

In [38]:
# Calculate total weight per X
total_weight_per_x = merged2.groupby('Order ID')['Weights (Kgs)'].sum().reset_index()
total_weight_per_x.rename(columns={'Weights (Kgs)': 'Total Weight as per X (Kg)'}, inplace=True)

# Drop any existing 'Total Weight as per X (KG)' column from merged2 DataFrame
if 'Total Weight as per X (Kg)' in merged2.columns:
    merged2.drop(columns=['Total Weight as per X (Kg)'], inplace=True)

# Merge the calculated total weight per X with the merged2 DataFrame
merged2 = pd.merge(merged2, total_weight_per_x, on='Order ID', how='left')

# Print the DataFrame to see the results
print(merged2.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8904223818706        1.0         127            173213   
1  2001827036  8904223819093        1.0         150            173213   
2  2001827036  8904223819109        1.0         100            173213   
3  2001827036  8904223818430        1.0         165            173213   
4  2001827036  8904223819277        1.0         350            173213   

  Type of Shipment  Warehouse Pincode Delivery Zone As Per X  Weights (Kgs)  \
0  Forward charges             121003                      e          0.127   
1  Forward charges             121003                      e          0.150   
2  Forward charges             121003                      e          0.100   
3  Forward charges             121003                      e          0.165   
4  Forward charges             121003                      e          0.350   

   Weight Slab As Per X  Expected Charge as per X  \
0                   0.5          

In [39]:
merged2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401 entries, 0 to 400
Data columns (total 15 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Order ID                                  401 non-null    int64  
 1   SKU                                       401 non-null    object 
 2   Order Qty                                 401 non-null    float64
 3   Weight (g)                                401 non-null    int64  
 4   Customer Pincode                          401 non-null    int64  
 5   Type of Shipment                          401 non-null    object 
 6   Warehouse Pincode                         401 non-null    int64  
 7   Delivery Zone As Per X                    401 non-null    object 
 8   Weights (Kgs)                             401 non-null    float64
 9   Weight Slab As Per X                      401 non-null    float64
 10  Expected Charge as per X              

In [40]:
merged_output = merged2.merge(courier_invoice, on='Order ID')
print(merged_output.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode_x  \
0  2001827036  8904223818706        1.0         127              173213   
1  2001827036  8904223819093        1.0         150              173213   
2  2001827036  8904223819109        1.0         100              173213   
3  2001827036  8904223818430        1.0         165              173213   
4  2001827036  8904223819277        1.0         350              173213   

  Type of Shipment_x  Warehouse Pincode_x Delivery Zone As Per X  \
0    Forward charges               121003                      e   
1    Forward charges               121003                      e   
2    Forward charges               121003                      e   
3    Forward charges               121003                      e   
4    Forward charges               121003                      e   

   Weights (Kgs)  Weight Slab As Per X  ...  \
0          0.127                   0.5  ...   
1          0.150                   0.5  ...   

In [41]:
merged_output.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode_x,Type of Shipment_x,Warehouse Pincode_x,Delivery Zone As Per X,Weights (Kgs),Weight Slab As Per X,...,Charges Billed by Courier Company,Total Weight as per X (Kg),AWB Code,Charged Weight,Warehouse Pincode_y,Customer Pincode_y,Delivery Zone Charged by Courier Company,Type of Shipment_y,Billing Amount (Rs.),Weight Slab Charged by Courier Company
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5,...,29.5,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5,...,29.5,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5,...,29.5,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5,...,29.5,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5,...,29.5,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0


In [42]:
merged_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401 entries, 0 to 400
Data columns (total 23 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Order ID                                  401 non-null    int64  
 1   SKU                                       401 non-null    object 
 2   Order Qty                                 401 non-null    float64
 3   Weight (g)                                401 non-null    int64  
 4   Customer Pincode_x                        401 non-null    int64  
 5   Type of Shipment_x                        401 non-null    object 
 6   Warehouse Pincode_x                       401 non-null    int64  
 7   Delivery Zone As Per X                    401 non-null    object 
 8   Weights (Kgs)                             401 non-null    float64
 9   Weight Slab As Per X                      401 non-null    float64
 10  Expected Charge as per X              

In [43]:
merged_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401 entries, 0 to 400
Data columns (total 23 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Order ID                                  401 non-null    int64  
 1   SKU                                       401 non-null    object 
 2   Order Qty                                 401 non-null    float64
 3   Weight (g)                                401 non-null    int64  
 4   Customer Pincode_x                        401 non-null    int64  
 5   Type of Shipment_x                        401 non-null    object 
 6   Warehouse Pincode_x                       401 non-null    int64  
 7   Delivery Zone As Per X                    401 non-null    object 
 8   Weights (Kgs)                             401 non-null    float64
 9   Weight Slab As Per X                      401 non-null    float64
 10  Expected Charge as per X              

In [44]:
merged_output.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode_x,Type of Shipment_x,Warehouse Pincode_x,Delivery Zone As Per X,Weights (Kgs),Weight Slab As Per X,...,Charges Billed by Courier Company,Total Weight as per X (Kg),AWB Code,Charged Weight,Warehouse Pincode_y,Customer Pincode_y,Delivery Zone Charged by Courier Company,Type of Shipment_y,Billing Amount (Rs.),Weight Slab Charged by Courier Company
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5,...,29.5,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5,...,29.5,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5,...,29.5,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5,...,29.5,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5,...,29.5,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0


In [45]:
merged_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401 entries, 0 to 400
Data columns (total 23 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Order ID                                  401 non-null    int64  
 1   SKU                                       401 non-null    object 
 2   Order Qty                                 401 non-null    float64
 3   Weight (g)                                401 non-null    int64  
 4   Customer Pincode_x                        401 non-null    int64  
 5   Type of Shipment_x                        401 non-null    object 
 6   Warehouse Pincode_x                       401 non-null    int64  
 7   Delivery Zone As Per X                    401 non-null    object 
 8   Weights (Kgs)                             401 non-null    float64
 9   Weight Slab As Per X                      401 non-null    float64
 10  Expected Charge as per X              

In [46]:
df_diff = merged_output
df_diff['Difference Between Expected Charges and Billed Charges(Rs.)'] = df_diff['Charges Billed by Courier Company'] - df_diff['Expected Charge as per X']

df_new = df_diff[['Order ID', 'Difference Between Expected Charges and Billed Charges(Rs.)', 'Expected Charge as per X']]

print(df_new.head())

     Order ID  Difference Between Expected Charges and Billed Charges(Rs.)  \
0  2001827036                                                0.0             
1  2001827036                                                0.0             
2  2001827036                                                0.0             
3  2001827036                                                0.0             
4  2001827036                                                0.0             

   Expected Charge as per X  
0                      29.5  
1                      29.5  
2                      29.5  
3                      29.5  
4                      29.5  


In [47]:
df_diff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401 entries, 0 to 400
Data columns (total 24 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Order ID                                                     401 non-null    int64  
 1   SKU                                                          401 non-null    object 
 2   Order Qty                                                    401 non-null    float64
 3   Weight (g)                                                   401 non-null    int64  
 4   Customer Pincode_x                                           401 non-null    int64  
 5   Type of Shipment_x                                           401 non-null    object 
 6   Warehouse Pincode_x                                          401 non-null    int64  
 7   Delivery Zone As Per X                                       401 non-null    obj

In [48]:
df_diff.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode_x,Type of Shipment_x,Warehouse Pincode_x,Delivery Zone As Per X,Weights (Kgs),Weight Slab As Per X,...,Total Weight as per X (Kg),AWB Code,Charged Weight,Warehouse Pincode_y,Customer Pincode_y,Delivery Zone Charged by Courier Company,Type of Shipment_y,Billing Amount (Rs.),Weight Slab Charged by Courier Company,Difference Between Expected Charges and Billed Charges(Rs.)
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5,...,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0,0.0
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5,...,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0,0.0
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5,...,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0,0.0
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5,...,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0,0.0
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5,...,2.039,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0,0.0


In [49]:
df_new = df_diff[['Order ID', 'AWB Code', 'Total Weight as per X (Kg)', 'Weight Slab As Per X','Total Weight as per Courier Company (KG)', 'Weight Slab Charge by Courier Company','Delivery Zone As Per X','Delivery Zone Charged by Courier Company','Expected Charge as per X','Charges Billed by Courier Company','Difference Between Expected Charges and Billed Charges(Rs.)']]

In [50]:
df_new.head()

,Order ID,AWB Code,Total Weight as per X (Kg),Weight Slab As Per X,Total Weight as per Courier Company (KG),Weight Slab Charge by Courier Company,Delivery Zone As Per X,Delivery Zone Charged by Courier Company,Expected Charge as per X,Charges Billed by Courier Company,Difference Between Expected Charges and Billed Charges(Rs.)
0,2001827036,1091122418320,2.039,0.5,0.0635,29.5,e,b,29.5,29.5,0.0
1,2001827036,1091122418320,2.039,0.5,0.0750,29.5,e,b,29.5,29.5,0.0
2,2001827036,1091122418320,2.039,0.5,0.0500,29.5,e,b,29.5,29.5,0.0
3,2001827036,1091122418320,2.039,0.5,0.0825,29.5,e,b,29.5,29.5,0.0
4,2001827036,1091122418320,2.039,0.5,0.1750,29.5,e,b,29.5,29.5,0.0


In [55]:
df_new.to_excel('Cointab_assignment.xlsx', index=False)  # Set index=False to exclude index from the Excel file


In [53]:
# Calculate the total orders in each category
total_correctly_charged = len(df_new[df_new['Difference Between Expected Charges and Billed Charges(Rs.)'] == 0])
total_overcharged = len(df_new[df_new['Difference Between Expected Charges and Billed Charges(Rs.)'] > 0])
total_undercharged = len(df_new[df_new['Difference Between Expected Charges and Billed Charges(Rs.)'] < 0])

# Calculate the total amount in each category
amount_overcharged = abs(df_new[df_new['Difference Between Expected Charges and Billed Charges(Rs.)'] > 0]['Difference Between Expected Charges and Billed Charges(Rs.)'].sum())
amount_undercharged = df_new[df_new['Difference Between Expected Charges and Billed Charges(Rs.)'] < 0]['Difference Between Expected Charges and Billed Charges(Rs.)'].sum()
amount_correctly_charged = df_new[df_new['Difference Between Expected Charges and Billed Charges(Rs.)'] == 0]['Expected Charge as per X'].sum()

# Create a new DataFrame for the summary
summary_data = {'Description': ['Total Orders where X has been correctly charged',
                                'Total Orders where X has been overcharged',
                                'Total Orders where X has been undercharged'],
                'Count': [total_correctly_charged, total_overcharged, total_undercharged],
                'Amount (Rs.)': [amount_correctly_charged, amount_overcharged, amount_undercharged]}

df_summary = pd.DataFrame(summary_data)

print(df_summary)

                                       Description  Count  Amount (Rs.)
0  Total Orders where X has been correctly charged    351       10425.3
1        Total Orders where X has been overcharged     50        1569.4
2       Total Orders where X has been undercharged      0           0.0


In [56]:
df_summary.to_excel('Summary.xlsx', index=False)  # Set index=False to exclude index from the Excel file